# Stabilities and accuracy in time. 

test

Lets consider same setup as in [ex_3a](https://github.com/AST-Course/AST5110/blob/main/ex_3a.ipynb) Burgers’ equation, i.e.,

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0   \tag{1}$$ 

for the domain $x \in (x_0, x_f)$ with $x_0 = −1.4$, $x_f = 2.0$ with initial condition:

$$u(x,t=0) = A\left[\tanh\left(\frac{x+x_c}{W}\right)-\tanh\left(\frac{x-x_c}{W}\right)\right]   \tag{2}$$

whereby $A = 0.02$ , $x_c = 0.70$, $W = 0.1$. Let the solution evolve until time $t_f = 100$. Explain in physical (or mathematical) terms the solution you get. However, let's now implement a new time-step method. 

Add the following method to your library and run the previous simulation. 

$$u^{n+1}_j = \frac{1}{2}(u^n_{j+1} + u^n_{j-1}) - \frac{v \Delta t}{2 \Delta x}(u^n_{j+1} - u^n_{j-1})$$

Apply von Neumann stability analysis to the expression above. The stability condition $|\xi(k)|^2 \leq 1$ leads to the famous Courant-Friedrichs-Lewy (CFL) stability. 

Add this to your library and solve the previous simulation imposing the CFL condition. For this exercise, fill in `nm_lib` the function `evolv_Lax_uadv_burgers`. 

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from nm_lib import nm_lib as nm
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [ ]:
def initial_u(xx):

    r"""
    Return the initial condition
    
    Requires
    ----------
    Numpy

    Parameters
    ----------
    x : `array`
        Spatial array

    Returns
    ------- 
    u_0(xx) : `array`
            Initial condition of u(x)
        
    """

    A = 0.02
    x_c = 0.7
    W = 0.1

    return A*(np.tanh((x+x_c)/W) - np.tanh((x-x_c)/W))

<!-- Eq.1 is the inviscid form of Burger's equation, which is a type of conservation equation. When evolving the solution in time, we see the edges are smoothed out.   -->

________

We implement the new time-step method:

$$\tag{3}
u^{n+1}_j = \frac{1}{2}(u^n_{j+1} + u^n_{j-1}) - \frac{v \Delta t}{2 \Delta x}(u^n_{j+1} - u^n_{j-1})
$$

Before implementing the method in our library, we apply the von Neumann stability analysis by substituting $u_j^{n} = \xi^n e^{ikj\Delta x}$. In order to have the CFL stability, the function must fulfill: $|\xi(k)|^2 \leq 1$. 



We start by writing out Eq.1 with our substitution: $u_j^{n} = \xi^n e^{ikj\Delta x}$: 

$$
\xi^{n+1}e^{ikj\Delta x} = \frac{1}{2}\left( \xi^n e^{ik(j+1)\Delta x} + \xi^n e^{ik(j-1)\Delta x}   \right) - \frac{v\Delta t}{2\Delta x} \left( \xi^n e^{ik(j+1)\Delta x} - \xi^n e^{ik(j-1)\Delta x}   \right)
$$

We can then split up the exponential functions and cancel common factors on each side. 

$$\tag{3.1}
\xi^{n+1}e^{ikj\Delta x} = \frac{1}{2}\left( \xi^n e^{ikj\Delta x}e^{ik\Delta x} + \xi^n e^{ik(j)\Delta x}e^{-ik\Delta x}   \right) - \frac{v\Delta t}{2\Delta x} \left( \xi^n e^{ikj\Delta x}e^{ik\Delta x} - \xi^n e^{ikj\Delta x}e^{-ik\Delta x}   \right)
$$

Cancelling the common factor $e^{ikj\Delta x}$ we are left with:

$$\tag{3.2}
\xi^{n+1} = \frac{1}{2}\left( \xi^n e^{ik\Delta x} + \xi^n e^{-ik\Delta x}   \right) - \frac{v\Delta t}{2\Delta x} \left( \xi^n e^{ik\Delta x} - \xi^n e^{-ik\Delta x}   \right)
$$

We can also rewrite the left-hand-side as $\xi^{n+1}e^{ikj\Delta x} = \xi^{n}\xi e^{ikj\Delta x}$. By cancelling the common factor $\xi^n$ we are left with:

$$\tag{3.3}
\xi = \frac{1}{2}\left( e^{ik\Delta x} +  e^{-ik\Delta x}   \right) - \frac{v\Delta t}{2\Delta x} \left(  e^{ik\Delta x} -  e^{-ik\Delta x}   \right)
$$

To simplify and cancel out more terms in Eq.3.3, we use the *Euler's formula* which applies to the exponential functions: 

$$
e^{ \pm i\theta } = \cos \theta \pm i\sin \theta
$$

Applying the Euler's formula, we can rewrite Eq.3.3 as

$$\tag{3.4}
\xi = \frac{1}{2} \left(  \cos(k\Delta x) + i\sin(k\Delta x) + \cos(k\Delta x) - i\sin(k\Delta x)  \right) - \frac{v\Delta t}{2\Delta x} \left( \cos(k\Delta x) + i\sin(k\Delta x) - (\cos(k\Delta x) - i\sin(k\Delta x))  \right)\\
= \cos(k\Delta x) - \frac{v\Delta t}{\Delta x} i\sin(k\Delta x)
$$

In Eq.3.4 we now have our simplified version of Eq.3, which we can use for the stability condition $|\xi(k)|^2 \leq 1$. We compute $|\xi(k)|^2$ by finding the square of the right-hand-side of Eq.3.4, using the complex conjugate:

$$\tag{4}
|\xi(k)|^2 = \left(  \frac{v\Delta t}{\Delta x} \right)^2 \sin^2 (k\Delta x) + \cos^2(k\Delta x)
$$

Using the *Pythagorean identity*, we can rewrite the cosine term, and hence Eq.4 becomes 

$$\tag{4.1}
|\xi(k)|^2 = \left(  \frac{v\Delta t}{\Delta x} \right)^2 \sin^2 (k\Delta x) + 1 +\sin^2(k\Delta x)\\
= \left( \left( \frac{v\Delta t}{\Delta x}  \right)^2 -1 \right)\sin^2(k \Delta x) + 1
$$

Since we want to check the stability condition $|\xi(k)|^2 \leq 1$, we use that $\sin^2 (k \Delta x) \leq 1$, meaning we get the inequality

$$\tag{4.2}
|\xi(k)|^2 \leq \left( \left( \frac{v\Delta t}{\Delta x}  \right)^2 -1 \right) + 1\\
= \left( \frac{v\Delta t}{\Delta x}  \right)^2 
$$

From Eq.4.2 we can therefore write 

$$
\left( \frac{v\Delta t}{\Delta x}  \right)^2 \leq 1 \implies \Delta t \leq \frac{\Delta x}{v}
$$

This shows our criteria for applying the CFL stability condition in our methods. We add this together with the method in Eq.3 in `nm_lib`, and use it to solve the previous simulation. For the simulation we use the same initial condition, `u0`, and spatial array `x`.

In [ ]:
nint = 128
nump = nint+1
x0 = -1.4
xf = 2

x = np.linspace(x0,xf, nump)
u0 = initial_u(x)

nt = 200
a = -1

t, un = nm.evolv_Lax_uadv_burgers(x, u0, nt, ddx = nm.deriv_cent, bnd_limits=[1,1])

In [ ]:
plt.ioff()

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

def init(): 
    axes.plot(x,un[:,0])

def animate(i):
 
    axes.clear()
    # axes.plot(x,un[:,::10][:,i])
    axes.plot(x,un[:,i])
    axes.set_title('t=%.2f'%t[i])
    axes.set_xlabel("x")
    axes.set_ylabel("u_i")
    axes.grid()
    
anim = FuncAnimation(fig, animate, interval=50, frames=nt, init_func=init)
HTML(anim.to_jshtml())

In [ ]:
plt.close()

In the animation above we show the simulation using the LAX method with the CFL stability condition. Eq.1 is known as the *inviscid* Burger's equation, which is known as a type of equation that can develop discontinuities, or shock waves. It seems as this is what we observe in the animation above, where we end up with a discontinuous wave shape that travels to the right. I think the shock wave is what we see as the vertical "wall" in the wave shape. 

## 1- Diffusive. 

Redo the exercise [ex_2b](https://github.com/AST-Course/AST5110/blob/main/ex_2b.ipynb) and compare the two methods, i.e., one from 
[ex_2b](https://github.com/AST-Course/AST5110/blob/main/ex_2b.ipynb) and the Lax-Method. 

Which one is more diffusive? Why? Rewrite Lax-method, so the right-hand side is as follows: 

$$\frac{u^{n+1}-u^{n}}{\Delta t} = ...$$

What is the reminder term look like? Is the order of convergence the same for the two methods? 

In [ ]:
nint = 128
nump = nint+1
x0 = -1.4
xf = 2

x = np.linspace(x0,xf, nump)
u0 = initial_u(x)

nt = 2000
a = -1

t_up, un_up = nm.evolv_uadv_burgers(x, u0, nt, ddx = nm.deriv_upw, bnd_limits=[1,0])
t, un = nm.evolv_Lax_uadv_burgers(x, u0, nt, ddx = nm.deriv_cent, bnd_limits=[1,0])

same = t_up.reshape(len(t_up),1) - t
same2 = np.where(np.abs(same)<0.5)


In [ ]:
plt.ioff()

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

def init(): 
    axes.plot(x,un_up[:,0])

def animate(i):
 
    axes.clear()
    axes.plot(x,un_up[:,::10][:,i], label="upwind")
    axes.set_title('t=%.2f'%t_up[::10][i])
    axes.legend()
    axes.set_xlabel("x")
    axes.set_ylabel("u_i")
    axes.grid()
    
anim = FuncAnimation(fig, animate, interval=50, frames=len(t_up[::10]), init_func=init)
HTML(anim.to_jshtml())

In [ ]:
plt.close()

In [ ]:
plt.ioff()

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

def init(): 
    axes.plot(x,un[:,0])

def animate(i):
 
    axes.clear()
    axes.plot(x,un[:,::10][:,i], label="LAX")
    axes.set_title('t=%.2f'%t[::10][i])
    axes.legend()
    axes.set_xlabel("x")
    axes.set_ylabel("u_i")
    axes.grid()
    
anim = FuncAnimation(fig, animate, interval=50, frames=len(t[::10]), init_func=init)
HTML(anim.to_jshtml())

In [ ]:
plt.close()

In [ ]:
plt.ioff()

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

def init(): 
    axes.plot(x,un[:,0])
    axes.plot(x,un_up[:,0])

def animate(i):
 
    axes.clear()
    axes.plot(x,un[:,same2[1]][:,::10][:,i], label="LAX")
    axes.plot(x,un_up[:,same2[0]][:,::10][:,i],label="upwind")
    axes.set_title('t=%.2f'%t[same2[1]][::10][i])
    axes.legend()
    axes.set_xlabel("x")
    axes.set_ylabel("u_i")
    axes.grid()
    
anim = FuncAnimation(fig, animate, interval=50, frames=len(t[same2[1]][::10]), init_func=init)
HTML(anim.to_jshtml())

In [ ]:
plt.close()

From the animation above, we see that the upwind scheme is the most diffusive one, as the amplitude is decreasing a lot faster than for the LAX method. To understand why, we rewrite the LAX-method so that the left-hand-side is the time derivative of $u$. 

Starting with the LAX method: 
$$u^{n+1}_j = \frac{1}{2}(u^n_{j+1} + u^n_{j-1}) - \frac{v \Delta t}{2 \Delta x}(u^n_{j+1} - u^n_{j-1})$$

We now subtract $u_j^n$ from each side

$$
u_{j}^{n+1} - u_j^n = \frac{1}{2} \left(  u_{j+1}^n + u_{j-1}^n \right) - u_j^n -\frac{v\Delta t}{2\Delta x} \left( u_{j+1}^n - u_{j-1}^n  \right),
$$

and divide by $\Delta t$

$$
\frac{u_{j}^{n+1} - u_j^n}{\Delta t} = \frac{\left(  u_{j+1}^n + u_{j-1}^n \right)}{2\Delta t} - \frac{u_j^n}{\Delta t} - \frac{v}{2\Delta x} \left( u_{j+1}^n - u_{j-1}^n.  \right)
$$

<!-- We can split up the $\frac{u_j^n}{\Delta t}$ term into two halfs, and subtract on half from each of the terms in $\frac{\left(  u_{j+1}^n + u_{j-1}^n \right)}{2\Delta t}$. This then becomes

$$
\frac{u_{j}^{n+1} - u_j^n}{\Delta t} =  \frac{\left(  u_{j+1}^n - u_{j}^n \right)}{2\Delta t} +  \frac{\left(  u_{j+1}^n - u_{j}^n \right)}{2\Delta t} - \frac{v}{2\Delta x} \left( u_{j+1}^n - u_{j-1}^n \right)
$$ -->

We can then gather the first three terms on the right-hand-side

$$
\frac{u_{j}^{n+1} - u_j^n}{\Delta t} = \frac{\Delta x^2 }{2\Delta t}\frac{\left(  u_{j+1}^n + u_{j-1}^n -2u_j^n \right)}{\Delta x^2} - \frac{v}{2\Delta x} \left( u_{j+1}^n - u_{j-1}^n \right),
$$

where we also multiplied and divided the gathered term with $\Delta x^2$. The term $\frac{\left(  u_{j+1}^n + u_{j-1}^n -2u_j^n \right)}{\Delta x^2}$ can be recognized as the second order spatial derivative of $u$. By also recognizing that the right-hand-side of the equation is the time derivative of $u$, while $\frac{1}{\Delta x} \left( u_{j+1}^n - u_{j-1}^n \right)$ is the first order spatial derivative of $u$ using centered finite differences, we can write the equation as

$$
\frac{\partial u}{\partial t} = \frac{\Delta x^2}{2\Delta t} \frac{\partial^2 u}{\partial^2 x} - v\frac{\partial u}{\partial x}
$$

We set $\frac{\Delta x^2}{2\Delta t} \equiv \nu$ and swap $v = u$. With this, we end up with:

$$
\frac{\partial u}{\partial t} = \nu \frac{\partial^2 u}{\partial^2 x} - u\frac{\partial u}{\partial x},
$$

which we can rewrite into the full Burger's equation

$$
\frac{\partial u}{\partial t} +  u\frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial^2 x}.
$$


From this, we can tell that the LAX method has an extra viscosity term, making it more resistance to diffusion than the upwind scheme. Therefore, the updwind scheme is more diffusive, which we see from how much faster it sinks in the animation.

---

(\*) Equation (1) is, in fact, a shortened version of the full Burgers’ equation, which contains a viscosity term on the right-hand side, as follows:
$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}.$$